# Module einlesen 

In [1]:
import pytesseract 
import matplotlib.pyplot as plt 
import cv2
import math
from PIL import Image
from collections import Counter 
from skimage.color import rgb2lab,deltaE_cie76,rgb2hsv
import os
from PIL import Image
import imutils
from sklearn.cluster import KMeans
import pandas as pd 
import numpy as np 
from pytesseract import Output
import scipy
from PIL import Image, ImageEnhance, ImageFilter
from skimage.filters import (threshold_otsu, threshold_niblack,
                             threshold_sauvola)
from skimage.filters import try_all_threshold
from skimage.filters import unsharp_mask

%matplotlib inline
# Pytesseract konfigurieren
pytesseract.pytesseract.tesseract_cmd=r'C:\Users\ashrafib\AppData\Local\Tesseract-OCR\tesseract.exe'

# Funktionen festlegen

## Text lesen 

In [2]:
def ocr_core(filename):
    """
    
    OCR - Preccessing definieren 
    
    """
    text=pytesseract.image_to_string(Image.open(filename),lang='deu')
    return text 

## RGB to HEX converter

In [3]:
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]),int(color[1]),int(color[2]))

## Extraktion der Hauptfarben 

In [7]:
def get_colors(pic,number_of_colors,show_chart):
    '''
    
    Input: 
    pic= bild 
    number_of_colors = Anzahl der Farben/ Nummer der Cluster bei kmeans 
    show_chart = bool zur Bilddarstellung
    
    '''
    ## Modifikation des Bildes     
    modified_image = cv2.resize(pic, (600, 400), interpolation = cv2.INTER_AREA)
    modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)
    
    ## Kmeans zur Extraktion der Farbcluster
    clf = KMeans(n_clusters = number_of_colors)
    labels = clf.fit_predict(modified_image)
    
    ## Counten der Labels/ Ordnen der Farben 
    counts = Counter(labels)   
    center_colors = clf.cluster_centers_
    ordered_colors = [center_colors[i] for i in counts.keys()]
    hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
    # Optional 
    rgb_colors = [ordered_colors[i] for i in counts.keys()]

    if (show_chart):
        plt.figure(figsize = (8, 6))
        plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

    return rgb_colors

# Main Funktion

In [12]:
def image_seg(): 
    '''
    Input: None
    Output: Bildsegmentierung von Pinke Post-ITs vorhanden
    Skript und Images müssen im gleichen Ordner liegen. 
    '''   
    folder=os.getcwd()+'\\'  
    id_pic=0
      
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
             
            id_pic += 1
            ls=[]
            ls.append(filename)

             # Boundaries Gelb
            lower_y = np.array([24,125,0])
            upper_y = np.array([50,255,255])
                
            for f in ls:
                img=cv2.imread(f)
                rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                
                result = img.copy()
                image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
                
                # Reduktion des Bildes auf Farbspektrum
                mask = cv2.inRange(image, lower_y, upper_y)
                result = cv2.bitwise_and(result, result, mask=mask)
                ims = cv2.resize(result, (960, 540))   
                
                cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
                cnts = imutils.grab_contours(cnts)
                
                #for c in cnts:
                # compute the center of the contour
                    #cv2.drawContours(result, [c], -1, (0, 255, 0), 2)
                    #cv2.imwrite('Image_1.png',result)
                
                path = folder +'ImageFolder'
                
                idx =0 
                for c in cnts:
                    idx += 1
                    x,y,w,h = cv2.boundingRect(c)
                    roi=result[y:y+h,x:x+w]
                    cv2.imwrite(os.path.join(path,str(id_pic)+'_'+str(idx) + '_f1.png'), roi) 
               
                for root,_, files in os.walk('C:\\Users\\ashrafib\\Desktop\\Projekte\\20_ImageSeg\\ImageFolder\\'):
                    for f in files: 
                        path = os.path.join(root, f)
                        try:
                            if os.path.getsize(path) < 100 * 1024:
                                os.remove(path)
                        except WindowsError:
                            print("Error" + path)
                    count=len(files)
                    print(count)